# Introduction


- In this project, we are investigating how different AI agents behave in a simple sealed-bid auction, which is where each agent gets a private value and has to decide how much to bid without seeing anyone else's bids. The main question here is *Given different goals and personalities, how do these agents choose their bids and what kinds of outcomes does that produce?*

- This is interesting from a computational social science perspective because auctions are everywhere (ads, markets, resource allocation, etc) and they are a clean way to study strategic behavior.

- In real auctions, people use shortcuts, worry about risk, try to outsmart each other, or just behave unpredictably. This project mixes AI-generated reasoning with actual computational tools like payoff calculators and simulations. The idea is to see what happens when you combine classical game-theoretic structure with LLM-style decision making. It ends up creating a small ecosystem where different agent types have to reason, compete, and adapt, giving us a more realistic picture than purely analytical models.



# System Design

The system is built around three main agent types running repeated sealed-bid auctions: an AuctioneerAgent that manages the environment, and two bidder types—StrategicBidderAgent and HeuristicBidderAgent—that take different approaches to bidding.

## The Agents

1. **The Auctioneer** is the coordinator.

    - Samples private values for each bidder from a configured distribution.

    - Collects bids, determines the winner and clearing price, and calls tools to compute utilities.

    - Sends back a compact outcome summary each round, which is also used later for evaluation and plots.
    
2. **The Strategic Bidder** is meant to represent a more “game-theoretic” participant:

    - It is roughly risk-neutral and tries to maximize expected utility.

    - It receives its private value and a simple belief model about how others bid.

    - It uses a best-response tool to scan through candidate bids, simulate many hypothetical auctions, and pick a bid that approximately maximizes expected utility.

3. **The Heuristic Bidder** is deliberately less sophisticated:

    - It follows simple rules like “shade my bid by a fixed factor” or “slightly overbid my value.”

    - It still has access to payoff calculations but does not run full search/optimization.

    - Over multiple auctions, it can update its shading parameter based on how often it wins and how much utility it gets, mimicking a rule-of-thumb learner instead of a full optimizer.

## The Tools
1. Payoff Calculator

    - The AuctioneerAgent uses this to finalize the outcome of a real auction round, and bidders can call it to ask: “If I bid X and others behave like this, what would my payoff be?”

2. Best-Response Approximator

    - The StrategicBidderAgent relies on this to turn “game-theoretic intent” into an actual numeric decision.

3. Auction Simulator

    - This simulator is mainly used by the Auctioneer. It runs many auctions in a row under fixed strategy profiles to get empirical stats (instead of just one-off outcomes).

## Interaction Protocol
The interaction pattern is a simple single-round loop that can be repeated for many auctions:

1. Auction Setup

    - The Auctioneer samples private values for all bidders from a specified distribution (e.g., uniform on [0, 1]).

    - It builds an AuctionConfig object (auction type, number of bidders, value distribution) and sends each bidder a BidRequest containing:

        - Their private value

        - The auction configuration

        - Optional history of past outcomes (for learning/adjustment)

2. Bid Generation

    - Each bidder processes the BidRequest with its own logic:

        - The StrategicBidderAgent calls the best-response tool to search over candidate bids.

        - The HeuristicBidderAgent applies its rule-of-thumb strategy, possibly updating its shading factor using past outcomes.

    - Each agent returns a BidResponse with: bidder_id, bid_amount,  A short reasoning_summary (for logging/interpretability).

    - Outcome Computation

        - The Auctioneer collects all BidResponses, decides the winner and price based on the auction type, and calls the payoff calculator.

        - The tool returns an AuctionOutcome with: winner_id, clearing_price, bids, values, payments, and per-agent payoff.

4. Feedback and Logging

    - The Auctioneer broadcasts a summary (who won, price, each bidder’s payoff) that can be stored for evaluation and optionally fed back into future BidRequest.history for learning.

Running this loop many times creates a dataset of auctions that you can analyze: distributions of bids, payoffs, win rates by agent type, and how behavior changes over time.

## Design Choices and Tradeoffs Discussion


### 1) LLM-Based Decision Making

All agent types utilize LLM reasoning.

Advantages:
* We gain the ability to understand the reasoning for each agents bid, rather than a hard-coded outcome.
* Ideally, each agent has the ability to learn from previous rounds and be more competitive.

Disadvantages:
* Much more expensive to have three LLM calls per round, for 20 rounds
* Runtime is much longer (not too overbearing for 20 rounds, but probably would not scale well)
* Lack of reproducibility
* LLM hallucination and reasoning failures can cause agents to behave unexpectedly

### 2) Allowing HeuristicAgent to dynamically change shading factor (0.7 - 0.95)

Even though the HeuristicAgent should follow a "rule-of-thumb" strategy, our simulation allows small adjustments based on history.

Advantages:
* Can see how reactive the agent is to positive or negative outcomes, if at all.
* Adds complexity and realism, preventing agents from being stuck.
  
Disadvantages:
* The LLM's interpretation of "when to adjust" may be unclear, even with reasoning provided.
* Over only 20 rounds, the agents tended to not show drastic adjustments.
* This functionality is the reason HeuristicAgent requires LLM interaction.

### 3) Limited to 3 Total Bidders

Advantages:
* Simpler
* Cheaper
* Faster

Disadvantages:
* May lose interesting insights and bidding trends when more factors are involved

### 4) Only Test Sealed-Bid Second-Price (Vickrey) Auction

We do not test other auction types (first-price, English, etc)

Advantages:
* Focus on one auction type for implementation and interpretation
* Simple payoff structure and auction setup

Disadvantages:
* May lose insights into how the different types of agents respond to different setups

### 5) What is included in "History"? 

The history that is passed to each agent each round contains:
* Previous Bids from all agents
* Previous Private Values from all agents
* Winner ID
* Round Index

Details not *explicitly* included (although maybe implicit)
* Payoffs
* Clearing Price
* Shading factors

Relying on the LLM to correctly infer all the omitted information could result in incorrect choices/reasonings, even if that information is implied.

# Implementation

#### 1. Core Data Models

#### 2. Core Game Logic

#### 3. Agents

In [ ]:
class BaseAgent(ABC):
    """
    Base class for all agents (Auctioneer + Bidders).

    - Provides shared prompt loading
    - Provides a placeholder call_llm you can later hook to OpenAI / other
    - Provides a helper to parse BidResponse JSON
    """

    def __init__(self, name: str):
        self.name = name

    @abstractmethod
    def get_bid(self, request: BidRequest) -> BidResponse:
        """
        For bidder agents: given a BidRequest, return a BidResponse.
        AuctioneerAgent won't implement this (it will have other methods).
        """

    def load_prompt_template(self, filename: str) -> str:
        """
        Load a prompt template from the prompts/ directory.
        """

    def call_llm(self, prompt: str, tools: Optional[list[Any]] = None) -> str:
        """
        Call the OpenAI Responses API with a simple prompt.

        Returns the *text* from the model's message, which we expect
        to be a JSON object string for bidders.
        """

    def _extract_json(self, text: str) -> dict[str, Any]:
        """
        Robustly extract JSON from LLM text.
        Assumes there is at least one {...} block.
        """
 
    def parse_bid_response(
        self, 
        request: BidRequest, 
        raw_text: str
    ) -> BidResponse:
        """
        Try to parse JSON from the model. If it fails, fall back to a
        simple BidResponse with the given fallback bid.
        """
 
class HeuristicBidderAgent(BaseAgent):
    """
    LLM-backed heuristic bidder.

    - Uses a simple shading rule as a fallback.
    - Optionally calls an LLM to wrap that rule in "reasoning".
    """

    def build_prompt(self, request: BidRequest, fallback_bid: float) -> str:
        if request.history:
            history_text = json.dumps(request.history, indent=2)
        else:
            history_text = "No history available."

        return f"""
        {self._shared}

        {self._persona}

        bidder_id: "{self.bidder_id}"
        private_value: {request.private_value}
        - current_shading_factor: {self.shading_factor}
        - fallback_bid (using current factor): {fallback_bid}

        Analyze carefully the history of past rounds, with winner_id and payoffs included:
        {history_text}

        Based on your performance in past rounds (wins, losses, payoffs), decide:
        1. Should you adjust your shading factor?
        2. What bid should you submit?
        3. Explain your reasoning for both decisions.

        Return JSON with:
        - "bid": your bid amount
        - "new_shading_factor": your adjusted factor (or keep current if no change)
        - "reasoning": explanation of your decision
        """.strip()

    def get_bid(self, request: BidRequest) -> BidResponse:
        """ return BidResponse """

class StrategicBidderAgent(BaseAgent):
    """
    LLM-backed strategic bidder.

    - Calls approximate_best_response() tool to get a recommended best bid.
    - Passes that recommendation into the LLM prompt.
    - LLM returns a JSON bid + reasoning.
    """

    def build_prompt(self, request: BidRequest, recommended_bid: float, expected_utility: float) -> str:
        if request.history:
            history_text = json.dumps(request.history, indent=2)
        else:
            history_text = "No history available."

        return f"""
            {self._shared}

            {self._persona}

            Information:
            - bidder_id: "{self.bidder_id}"
            - private_value: {request.private_value}
            - best_response_calculator_recommendation:
                - best_bid: {recommended_bid}
                - expected_utility: {expected_utility}

            history: "{history_text}"
            """.strip()

    def get_bid(self, request: BidRequest) -> BidResponse:
        """ return BidResponse """

### Tool Defs

In [ ]:
###### Payoff Calculator ##########
def compute_payoffs(
    bids: Dict[str, float],
    values: Dict[str, float],
    auction_id: str = "simulated_auction",
    round_index: int = 0,
) -> AuctionOutcome:
    """
    Run a second-price auction and compute quasilinear payoffs.

    Assumes:
    - run_second_price_auction(bids) -> (winner_id, price)
    - Payoff for winner i:  u_i = v_i - price
    - Payoff for losers:    u_j = 0
    """

###### Best Response Calc ############
def approximate_best_response(private_value: float, num_grid_points: int = 101) -> dict:
    """
    Approximate a best-response bid in a second-price auction against
    (by default) truthful, uniformly distributed opponents in [0, 1].

    Uses the payoff calculator internally to evaluate expected utility.
    """

####### Auction Simulator ###########
def run_simulation(
    auction_config: AuctionConfig,
    strategy_profiles: Dict[str, str],
    num_rounds: int,
) -> SimulationSummary:
    """
    Run many auctions under simple hard-coded strategies.

    strategy_profiles: bidder_id -> strategy name
        strategy in {"truthful", "shaded_0.8"}
    """

### State Management

In [ ]:
class AuctioneerAgent(BaseAgent):
    """
    Coordinates one auction round:
    - Samples private values
    - Asks each bidder for a BidResponse
    - Calls payoff calculator to get AuctionOutcome
    """

    def get_bid(self, request: BidRequest) -> BidResponse:
        """ Auctioneer does not bid; this method is not used. """
        
    def run_round(self, bidders: Sequence[BaseAgent]) -> AuctionOutcome:
        """ Run a single sealed-bid second-price auction round. """
       
    def generate_round_summary(self, outcome: AuctionOutcome):
        """ Generates an LLM-written human-readable summary for a single round, not entire auction. """

    def build_history(self, exclude_round: int = None) -> dict:
        """ Return a structured history, optionally excluding a specific round. """


# Experiments + Results

## Experiments Breakdown

Five unique combinations of agents/parameters were used to observe behavioral differences.

1) 'Default' scenario, 2 heuristic and 1 strategic agent
2) 3 heuristic agents
3) 'Default' scenario, but starting the heuristic agents at more conservative shading factors
4) 3 strategic agents
5) 2 strategic agents, 1 heuristic agent

Each scenario:
* Contains a round-by-round summary in /scenario_logs/scenario_X_log.txt, where LLM generated reasonings and summarizations are provided.
* Uses 20 rounds
* Has the agents follow the same prompts/rules/tools across simulations.
* Each bidder receives a private value sampled uniformly from [0,1].
* All use the same random seed

--------------

### Scenario 1

- 'Default', one Strategic agent and two Heuristic Agents

### Output

Running 20 LLM-based auctions...

=== LLM Auction Simulation Summary ===

Mean Clearing Price (Revenue): 0.3829

Mean Utility per Bidder:
  • B1: 0.1232
  • B2: 0.1489
  • B3: 0.0887

Winner Distribution:
  • B1: 35.0%
  • B3: 25.0%
  • B2: 40.0%

======================================

### Plots

![](viz/scenario1/Figure_1.png)
![](viz/scenario1/Figure_2.png)
![](viz/scenario1/Figure_3.png)
![](viz/scenario1/Figure_4.png)

#### Scenario 1 Evaluation

Despite having access to the best-response tool, the Strategic agent (B2) only wins one more time than the next best heuristic competitor (B1), and collects slightly more utility on average per round. In theory, we might've expected the Strategic agent to have an advantage over the more rule-based competitors, but they may not be effectively adapting to the situation. Additionally, the strategic agents rarely bid at/above their secret value, showing they may not adapt to the Vickrey theory that your best option is to bid your secret value.

------
### Scenario 2

* 3 heuristic agents, with 0.9, 0.8, 0.7 inital shading factors
* see scenario_logs/scenario_2_log.txt
* see viz/scenario2/Figure_X.png

### Output

Running 20 LLM-based auctions...

=== LLM Auction Simulation Summary ===

Mean Clearing Price (Revenue): 0.4307

Mean Utility per Bidder:
  • B1: 0.1800
  • B2: 0.1251
  • B3: 0.0426

Winner Distribution:
  • B1: 55.0%
  • B2: 30.0%
  • B3: 15.0%

======================================

### Plots

![](viz/scenario2/Figure_1.png)
![](viz/scenario2/Figure_2.png)
![](viz/scenario2/Figure_3.png)
![](viz/scenario2/Figure_4.png)

#### Scenario 2 Evaluation

This scenario displays the 2nd highest average clearing price among our five scenarios, implying that using rule-based bidders is more competitive. B1, the bidder who starts at 0.9 shading factor, ends up dominating this simulation, which may be expected. This agent does not begin to adjust its shading factor until one of the last rounds, potentially identifying that staying at a close to true value bid was working more often than not. Unsurprisingly, the bidders finished in order of their starting shading factor. It may have been expected that B2 and B3 (who started lower) would increase their shading factor at a quicker rate to try and produce more wins. This could be a shortcoming with the prompts, not incentivizing this change enough, or the LLM agent was content with "not losing" (i.e. 0 utility) rather than winning.


-----------
### Scenario 3

- One StrategicBidderAgent and two HeuristicBidderAgents (start at lower shading factors).
- see scenario_logs/scenario_3_log.txt
- see viz/scenario_3/Figure_X.png for each visual

### Output

Running 20 LLM-based auctions...

=== LLM Auction Simulation Summary ===

Mean Clearing Price (Revenue): 0.3563

Mean Utility per Bidder:
  • B1: 0.1036
  • B2: 0.1084
  • B3: 0.1796

Winner Distribution:
  • B3: 35.0%
  • B1: 25.0%
  • B2: 40.0%

======================================

### Plots

![](viz/scenario3/Figure_1.png)
![](viz/scenario3/Figure_2.png)
![](viz/scenario3/Figure_3.png)
![](viz/scenario3/Figure_4.png)

#### Scenario 3 Evaluation

B2 (strategic) and B3 (heuristic) perform similarly in terms of wins, while B2 and B1 (other heuristic) see a similar average utility gain. This simulation contains the lowest average clearing price, which is to be expected given the lower starting shading factors. B3, who maintains a slightly higher increase to their shading factor, collects by far the most average utility, showing that a more 'true' bid amongst heuristic agents could be more beneficial than shading lower. Given the adjustment to the heuristic agent starting points, it may have been expected that the strategic agent could perform better to take advantage, however, the strategic agent shows signs of following the calculator recommendation rather than adapting to the heuristic agents' behavior.


-----------
### Scenario 4

- Three StrategicAgents
- see scenario_logs/scenario_4_log.txt
- see viz/scenario_4/Figure_X.png for each visual
    - Shading_Factor history not available since all 3 are strategic

### Output

Running 20 LLM-based auctions...

No shading factor data available to plot.

=== LLM Auction Simulation Summary ===

Mean Clearing Price (Revenue): 0.3960

Mean Utility per Bidder:
  • B1: 0.0846
  • B2: 0.1410
  • B3: 0.0417

Winner Distribution:
  • B1: 30.0%
  • B3: 25.0%
  • B2: 45.0%

======================================

### Plots

![](viz/scenario4/Figure_1.png)
![](viz/scenario4/Figure_2.png)

![](viz/scenario4/Figure_4.png)

#### Scenario 4 Evaluation

In each round of this simulation, most of the winnings bids are at or 'generally' around the winners secret value, which, according to Vickrey auction theory, is the true best response, however, the average utility for each player is the lowest among our simulations! Depending on your definition of "close", at least 13/20 rounds show the victor bidding very close to their secret value. In our setup, even though the agents *may* be finding the right intution with the help of the best response calculator, this actually results in a less profitable market for all involved. Thinking about how heuristic vs strategic agents work in our setup, we theortically expect a strategic agent to bid higher than a heuristic one (given the same secret value), therefore, the difference between a winning bid and the second bid compresses when all agents are strategic, compressing payoffs. That is reflected in the results here. Also, the majority of the 'reasonings' provided by the agents note they are just following the calculators output, showing no real signs of learning throughout the auction.


-----------
### Scenario 5

- Two StrategicBidderAgents (B2, B3) and one HeuristicBidderAgent (B1)
- see scenario_logs/scenario_5_log.txt
- see viz/scenario_5/Figure_X.png for each visual

### Output

Running 20 LLM-based auctions...

=== LLM Auction Simulation Summary ===

Mean Clearing Price (Revenue): 0.4379

Mean Utility per Bidder:
  • B1: 0.0859
  • B2: 0.1428
  • B3: 0.0757

Winner Distribution:
  • B2: 50.0%
  • B1: 20.0%
  • B3: 30.0%

======================================

### Plots

![](viz/scenario5/Figure_1.png)
![](viz/scenario5/Figure_2.png)
![](viz/scenario5/Figure_3.png)
![](viz/scenario5/Figure_4.png)

#### Scenario 5 Evaluation

In this 2 vs 1 setting, the strategic agents (B2, B3) dominate the lone heuristic agent in terms of winning. The majority of the wins from the strategic agents come when they bid at or around their true secret value, following the theoretical expectation. When in competition with multiple strategic agents, it appears our rule-based agent cannot win nearly as many rounds, even if its average utility was comprable to one of the strategic agents. The heuristic agent does not change their shading factor at any point, likely to its detriment. Notably, B2 dominates B3 in both wins and utility, even with both having the same tools, which could be due to the luck of the private value generation.

## Test Cases Breakdown

Three different test cases were created to evaluate the logic behind the auctions. 

1. Removed "History" component from all agents
2. Removed the LLM aspect from all agents
3. Simplfied agents' prompts to be as minimal as possible.

### Test Case 1: No History

#### Output

=== LLM Auction Simulation Summary ===

Mean Clearing Price (Revenue): 0.3809

Mean Utility per Bidder:
  • B1: 0.1890
  • B2: 0.0717
  • B3: 0.0547

Winner Distribution:
  • B1: 55.0%
  • B2: 35.0%
  • B3: 10.0%

======================================

#### Plots 

![](tests/nohistory/Figure_1.png)
![](tests/nohistory/Figure_2.png)
![](tests/nohistory/Figure_3.png)
![](tests/nohistory/Figure_4.png)

### Test Case 2: No LLM

#### Output

=== LLM Auction Simulation Summary ===

Mean Clearing Price (Revenue): 0.4849

Mean Utility per Bidder:
  • B1: 0.0932
  • B2: 0.1603
  • B3: 0.0708

Winner Distribution:
  • B2: 60.0%
  • B3: 15.0%
  • B1: 25.0%

======================================

#### Plots

![](tests/noLLM/Figure_1.png)
![](tests/noLLM/Figure_2.png)
![](tests/noLLM/Figure_3.png)
![](tests/noLLM/Figure_4.png)

### Test Case 3: Minimal Prompt

#### Output


Mean Clearing Price (Revenue): 0.4249

Mean Utility per Bidder:
  • B1: 0.0632
  • B2: 0.0747
  • B3: 0.1366

Winner Distribution:
  • B1: 15.0%
  • B2: 45.0%
  • B3: 40.0%

======================================

#### Plots

![](tests/simpleprompt/Figure_1.png)
![](tests/simpleprompt/Figure_2.png)
![](tests/simpleprompt/Figure_3.png)
![](tests/simpleprompt/Figure_4.png)

# Analysis + Discussion

Interpret your results

Connect to course concepts (game theory, emergence, networks, equilibria)

Discuss limitations and future extensions

Reflect on what you learned


# Conclusion

Summarize key findings

Implications for computational social science

* With current implementation, the strategic agent doesnt really "learn". It has access to the best response (calculated and passed to LLM), but even with auction history, never questions why it isnt winning more and never learns the optimal strategy of always bidding your true value in a Vickrey style auciton.